In [162]:
import pandas as pd
pd.set_option('display.max_rows', 500)
import os

In [163]:
PATH = "../00_data/0_raw/"
all_files = [f for f in os.listdir(PATH)] #get all file names

#use file names as column names for later
def new_col_names(name):
    #print(os.path.basename(name).split('.')[0].replace('-', '_'))
    return os.path.basename(name).split('.')[0].replace('-', '_')

#make label u5mr first
label_file = all_files.pop(5)
all_files.insert(0, label_file)

In [178]:
big_df = None
joins = ['Entity', 'Code', 'Year']

EXCLUDE_NO_COUNTRIES = ["Africa", "Asia", "Europe", "European Union (27)", "High-income countries", "Low-income countries", "Lower-middle-income countries", 
                      "North America", "Oceania", "South America", "Upper-middle-income countries", "World"]

#change column names and add as columns to df
for name in all_files:
    cols_names = new_col_names(name)

    df = pd.read_csv(os.path.join(PATH, name), usecols=[0, 1, 2, 3])
    df.columns = joins + [cols_names]

    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html
    df = df[~df['Entity'].isin(EXCLUDE_NO_COUNTRIES)] #remove continents etc. from df
    
    df = df.set_index(joins) #entity, code, year as index

    if big_df is None:
        big_df = df.copy() 
    else: #outer join - merge all dfs
        big_df = big_df.merge(
            df, 
            left_index=True, 
            right_index=True, 
            how='outer' 
        )

big_df.loc[("Ghana")].head(15)
big_df.info()
#big_df.shape
big_df.isnull().sum()
#big_df.head(450)

child_mortality_igme  \
Entity      Code Year                         
Afghanistan AFG  1957                 False   
                 1958                 False   
                 1959                 False   
                 1960                 False   
                 1961                 False   
...                                     ...   
Zimbabwe    ZWE  2020                 False   
                 2021                 False   
                 2022                 False   
                 2023                 False   
                 2024                  True   

                       nurses_and_midwives_per_1000_people  \
Entity      Code Year                                        
Afghanistan AFG  1957                                 True   
                 1958                                 True   
                 1959                                 True   
                 1960                                 True   
                 1961                                 True   
...                                                    ...   
Zimbabwe    ZWE  2020                                False   
                 2021                                 True   
                 2022                                False   
                 2023                                 True   
                 2024                                 True   

                       annual_healthcare_expenditure_per_capita  \
Entity      Code Year                                             
Afghanistan AFG  1957                                      True   
                 1958                                      True   
                 1959                                      True   
                 1960                                      True   
                 1961                                      True   
...                                                         ...   
Zimbabwe    ZWE  2020                                     False   
                 2021                                     False   
                 2022                                     False   
                 2023                                      True   
                 2024                                      True   

                       gdp_per_capita_worldbank  share_of_population_urban  \
Entity      Code Year                                                        
Afghanistan AFG  1957                      True                       True   
                 1958                      True                       True   
                 1959                      True                       True   
                 1960                      True                      False   
                 1961                      True                      False   
...                                         ...                        ...   
Zimbabwe    ZWE  2020                     False                      False   
                 2021                     False                      False   
                 2022                     False                      False   
                 2023                     False                      False   
                 2024                     False                      False   

                       years_of_schooling  physicians_per_1000_people  \
Entity      Code Year                                                   
Afghanistan AFG  1957                True                        True   
                 1958                True                        True   
                 1959                True                        True   
                 1960                True                       False   
                 1961                True                        True   
...                                   ...                         ...   
Zimbabwe    ZWE  2020               False                       False   
                 2021               False                   

In [123]:
#u5mr_df = pd.read_csv("../00_data/0_raw/child-mortality-igme.csv")
#u5mr_df.head()